In [ ]:
import sys

from pyspark.sql import SparkSession

# Проверка доступа к Hadoop

Hadoop стартует в однонодовом режиме: все демоны на одном компьютере. Демоны Hadoop:

- демон [NameNode](http://localhost:9870) запущен на `9870` порту,
- демон [DataNode](http://localhost:9864) запущен на `9864` порту,
- демон [ResourceManager](http://localhost:8088) запущен на `8088` порту порту,
- демон [NodeManager](http://localhost:8042) запущен на `8042` порту,

# Настройка количества воркеров

## Статическое число воркеров

Число контейненров для воркеров можно настраивать через `spark.executor.instances` (по умолчанию 2):

In [ ]:
spark = (
    SparkSession
        .builder
        .appName("Yarn Static Workers")
        .master("yarn")
        .config("spark.executor.instances", 3)
        .getOrCreate()
)
sc = spark.sparkContext

Yarn выделит один контейнер для драйвера (Application Master) и `spark.executor.instances` контейнеров для воркеров:

In [ ]:
! source ~/.bash_aliases && \
yarn application -list

In [ ]:
! source ~/.bash_aliases && \
yarn container -list $( \
    yarn applicationattempt -list $( \
        yarn application -appTypes SPARK -appStates RUNNING -list | awk '/application_/{print $1}' \
    ) | awk '/appattempt_/{print $1}' \
)

Как и ожидалось, команда выше показывает, что для текущего приложения запущено 4 контейнера: `3 (spark.executor.instances) + 1`

In [ ]:
spark.stop()

## Динамическое число воркеров

Число контейнеров Yarn выделенных Spark приложению остается неизменным. Но для интерактивных нагрузок, потребность в ресурсах может появляться и исчезать, а значит приложение может отпускать ресурсы, чтобы другие приложения могли ими воспользоваться или запросить больше ресурсов, для более быстрого исполнения больших задач. На этот случай Apache Spark позволяет динамически запрашивать ресурсы в кластере.

### Способ активации

По умолчанию динамическое выделение ресурсов отключено на стороне Apache Spark. Для включения необходимо установить значение `True` в `spark.dynamicAllocation.enabled`.

Дополнительно необходимо указать сколько контейнеров должно быть доступно всегда `spark.dynamicAllocation.minExecutors` (по умолчанию 0) и сколько можно запросить контейнеров всего `spark.dynamicAllocation.maxExecutors` (не ограничено)

Если воркер не выполняет ниакой работы в течении `spark.dynamicAllocation.executorIdleTimeout` секунд (по умолчанию 60), то воркер удаляется.

Если на воркере закешированы данные, то Apache Spark не будет его удалять, даже если он находится без работы дольше чем `spark.dynamicAllocation.executorIdleTimeout`. Для установки времени жизни воркеров с закешированными данными необходимо указать значение в секундах в `spark.dynamicAllocation.cachedExecutorIdleTimeout` (по умолчанию, бесконечно)

Указанные настройки должны быть установлены во время старта приложения и не могут меняться на всем протяжении жизни приложения:

In [ ]:
spark = (
    SparkSession
        .builder
        .appName("Yarn Dymanic Workers")
        .master("yarn")
        .config("spark.dynamicAllocation.enabled", True)
        .config("spark.dynamicAllocation.minExecutors", 1)
        .config("spark.dynamicAllocation.maxExecutors", 4)
        .config("spark.dynamicAllocation.executorIdleTimeout", 20)
        .config("spark.dynamicAllocation.cachedExecutorIdleTimeout", 60)
        .getOrCreate()
)
sc = spark.sparkContext

После старта приложения Yarn выделит один контейнер для драйвера (Application Master) и `spark.dynamicAllocation.maxExecutors` контейнеров для воркеров:

In [ ]:
! source ~/.bash_aliases && \
yarn application -list

In [ ]:
! source ~/.bash_aliases && \
yarn container -list $( \
    yarn applicationattempt -list $( \
        yarn application -appTypes SPARK -appStates RUNNING -list | awk '/application_/{print $1}' \
    ) | awk '/appattempt_/{print $1}' \
)

Но по прошествии `spark.dynamicAllocation.executorIdleTimeout` останется один контейнер для драйвера (Application Master) и `spark.dynamicAllocation.minExecutors` контейнеров для воркеров:

Если подождать 20 секунд с момента старта приложения перед запуском следующей команды, то можно увидеть, что для текущего приложения запущено 2 контейнера: `1 (spark.dynamicAllocation.minExecutors) + 1`

In [ ]:
! source ~/.bash_aliases && \
yarn container -list $( \
    yarn applicationattempt -list $( \
        yarn application -appTypes SPARK -appStates RUNNING -list | awk '/application_/{print $1}' \
    ) | awk '/appattempt_/{print $1}' \
)

In [ ]:
spark.stop()

## Вывод

Apache Spark позволяет гибко настроить приложение под любой тип нагрузки: как для интерактивной аналитики с динамическим выделением ресурсов, так и батчевой или стримовой нагрузки со статическим выделением ресурсов.

## Задания

1. Запустить Apache Spark приложение с настройками по умолчанию. Сколько контейнеров Yarn выделяет такому приложению?
2. Запустить Apache Spark приложение с динамическим выделением ресурсов в соответствии со следующими требованиями:
    - в ждушем режиме не должно быть ниодного контейнера для воркеров,
    - максимальное число контейнеров 3,
    - время работы контейнера в ждущем режиме две минуты.
3. Запустить приложение с динамическим выделением ресурсов из предыдущего шага. При этом время жизни воркеров в ждущем режиме с закешированными данными должно не превышать 2 минуты 30 секунд. Поставить эксперимент и проверить на практике.

<details>
    <summary>Нужна помощь?</summary>

    Ответы внизу страницы
</details>

# Настройка памяти

## Выделение памяти в Yarn

Программист определяет объем памяти:

- необходимый драйверу:
    - в кластерном режиме через настройку `spark.driver.memory` (1GB по умолчанию),
    - в клиентском режиме через настройку `spark.yarn.am.memory` (512MB по умолчанию),
- необходимый воркеру через настройку `spark.executor.memory` (1GB по умолчанию). При этом каждому воркеру будет выделен объем памяти указанный в `spark.executor.memory`.

**Внимание**: PySpark в ноутбуке можно запустить только в клиентском режиме!

> PySpark работает только в клиентском режиме!

Yarn выделяет ресурсы слотами: один слот содержит минимальный объем памяти. В соответствии с настройками приложение Apache Spark запрашивает определенный объем памяти, а Yarn выдаст число слотов, которое будет покрывать запрощенный объем.

Пример:

Дано:

- минимальный размер памяти: 1024MB
- объем памяти для воркера 1500MB

Найти: объем памяти одного воркера.

Решение:

Одного слота недостаточно, т.к. 1024MB < 1500MB, поэтому yarn выделит 2 слота. Итого у каждого воркера будет по $ 2 \space слота \times 1024MB \space (минимальный \space размер \space памяти) = 2048MB$.

### Задание

Сколько памяти выделит Yarn на запрос `32MB`, если минимальный размер слота раверн `512MB`?

<details>
    <summary>Ответ</summary>

    выделится 1 слот, в котором 512MB (минимальный размер слота)
</details>

### Максимальный объем

Yarn может обрабтать не любой запрос. Так, если запрос некоторое превышает пороговое значение, то Yarn откажет в обработке такого запроса.

Пороговое значение по умолчанию для Yarn является 8GB, даже при наличии свободных слотов Yarn не будет выпделять память за пределами этого значения.

## Настройка Yarn

[Настройка Yarn](https://hadoop.apache.org/docs/stable/hadoop-yarn/hadoop-yarn-common/yarn-default.xml) выполняется через `yarn-site.xml` файл. В частности:

- минимальный размер памяти в слоте настраивается через `yarn.scheduler.minimum-allocation-mb`,
- максимальный размер памяти настраивается через `yarn.scheduler.maximum-allocation-mb`.

В текущей конфигурации:

In [ ]:
! curl -s http://${RESOURCEMANAGER_HOST}:8088/conf | grep 'yarn.scheduler.minimum-allocation-mb' | xmllint --format -

In [ ]:
! curl -s http://${RESOURCEMANAGER_HOST}:8088/conf | grep 'yarn.scheduler.maximum-allocation-mb' | xmllint --format -

## Запрос ресурсов

Для проверки выделения ресурсов следующее приложение стартует с

- одним контейнером для драйвера (Application Master),
- тремя контейнерами для воркеров.

In [ ]:
spark = (
    SparkSession
        .builder
        .appName("Yarn")
        .master("yarn")
        .config("spark.executor.instances", 3)
        .config("spark.yarn.am.memory", "512m")
        .config("spark.executor.memory", "1g")
        .getOrCreate()
)
sc = spark.sparkContext

Следующая команда покажет запущенное Apache Spark приложение:

In [ ]:
! source ~/.bash_aliases && \
yarn application -list

Проверим, что действительно запущено 4 контейнера:

In [ ]:
! source ~/.bash_aliases && \
yarn container -list $( \
    yarn applicationattempt -list $( \
        yarn application -appTypes SPARK -appStates RUNNING -list | awk '/application_/{print $1}' \
    ) | awk '/appattempt_/{print $1}' \
)

Можно ожидать, что всего будет выделено $ 1 \space драйвер \times 1G (слот \space для \space spark.yarn.am.memory) + 3 \space воркера \times 1G (spark.driver.memory) = 4G = 4096MB$

Текущий сетап кластера Hadoop состоит из одного узла, который одновременно играет все 4 роли:

- NameNode,
- DataNode,
- ResourceManager,
- NodeManager.

Поэтому можно заключить, что ресурсы выделенные на NodeManager - это все ресурсы выделенные на приложение Apache Spark:

In [ ]:
! source ~/.bash_aliases && \
yarn node -all -list -showDetails

Команда выше показывает, что выделено всего `7168MB` (Allocated Resources), хотя ожидалось, что будет `4096MB`.

Откуда разница почти в $7168MB - 4096MB \approx 3G$?

Можно предположить, что Hadoop занимает память какими-то своими служебными процессами. Но если остановить приложение Spark:

In [ ]:
spark.stop()

То объем выделенных ресурсов будет равен нулю:

In [ ]:
! source ~/.bash_aliases && \
yarn node -all -list -showDetails

Команда выше демонстрирует, что ни памяти, ни ядер процессора не выделено: `Allocated Resources : <memory:0, vCores:0>`

В приложении с тремя воркерами разница в три гигабайта намекает на то, что каждый воркер забрал себе по одному дополнительному гигабайту.

Для продолжения экспериментов запустим Spark приложение с предыдущей конфигурацией:

- 1 драйвер,
- 3 воркера,
- 512MБ на драйвер,
- 1ГБ на воркер.

In [ ]:
spark = (
    SparkSession
        .builder
        .appName("Yarn")
        .master("yarn")
        .config("spark.executor.instances", 3)
        .config("spark.yarn.am.memory", "512m")
        .config("spark.executor.memory", "1g")
        .getOrCreate()
)
sc = spark.sparkContext

Запущенное приложение

In [ ]:
! source ~/.bash_aliases && \
yarn application -appTypes SPARK -appStates RUNNING -list

Если посмотреть в логи, то можно увидеть какие ресурсы и в каком объеме запрашивал драйвер для воркеров:

In [ ]:
! source ~/.bash_aliases && \
yarn logs -am -1 -log_files stderr -applicationId $( \
    yarn application -appTypes SPARK -appStates RUNNING -list | awk '/application_/{print $1}' \
) | grep YarnAllocator

В выводе выше видно, что драйвер запрашивает:

- три контейнера: `Will request 3 executor container(s)`;
- `1408MB` объем памяти для одного контейнера: `with custom resources: <memory:1408...`;
- одно ядро для контейнера: `with custom resources: <..., vCores:1>`.

Учитывая, что минимальный объем памяти в слоте равен `1024MB`, `Yarn` ничего не остается как выделить по два слота на контейнер, поэтому каждый воркер получает по `2` гигабайта памяти.

Тогда всё сходится: $ 1024MB (слот \space для \space spark.yarn.am.memory) + 3 \space (spark.executor.instances) \times 1024 (минимальный \space размер \space слота) \times 2 (слота) = 7168MB $

**Внимание**: Помним, что хотя `spark.yarn.am.memory` равен `512MB`, но Yarn не может выделить меньше `1024MB` за раз.

In [ ]:
! source ~/.bash_aliases && \
yarn node -all -list -showDetails 2>/dev/null | grep 'Allocated Resources'

In [ ]:
spark.stop()

### Задание

1. Аналитически определить сколько памяти будет выделено на аналогичное приложение с двумя воркерами.
2. Запустить приложение с двумя воркерами. Проверить, что теоретические и практические значения совпадают.

<details>
    <summary>Нужна помощь?</summary>

    Ответы внизу страницы
</details>

## Off-Heap память

In [ ]:
spark.stop()

Так откуда же берётся число `1408` при запросе контейнеров для воркеров?

Помимо указания `spark.executor.memory` присутствует еще одна настройка `spark.executor.memoryOverhead`, которая указывает сколько памяти воркер будет использовать для Off-Heap памяти. Off-Heap память используется для хранния стек-трейсов потоков, прямых операциях с памятью, кешировании (при указании `StorageLevel.OFF_HEAP`) и других структур данных, которые воркер создает вне общей памяти. Off-Heap память управляется программистом напрямую и не участвует в сборке мусора.

По умолчанию ее объем равен 10% от `spark.executor.memory`, но не меньше `384MB`. Так $ 1024MB (spark.memory.executor) + 384MB (spark.executor.memoryOverhead) = 1408MB $, ровно то самое число, которое Spark Application Master запрашивает у Yarn при создании воркеров.

In [ ]:
spark = (
    SparkSession
        .builder
        .appName("Yarn")
        .master("yarn")
        .config("spark.executor.instances", 3)
        .config("spark.yarn.am.memory", "512m")
        .config("spark.executor.memory", "1g")
        .config("spark.executor.memoryOverhead", "500m")
        .getOrCreate()
)
sc = spark.sparkContext

Проверим какие ресурсы и в каких объемах Spark запрашивал при создании воркеров:

In [ ]:
! source ~/.bash_aliases && \
yarn logs -am -1 -log_files stderr -applicationId $( \
    yarn application -appTypes SPARK -appStates RUNNING -list |& awk '/application_/{print $1}' \
) |& grep 'YarnAllocator: Will request'

В логах теперь значится ожидаемое значение `1524`: $ 1024MB (spark.executor.memory) + 500MB (spark.executor.memoryOverhead) = 1524MB $

In [ ]:
spark.stop()

### Задание

1. Аналитически вычислить объем оперативной памяти для аналогичного приложения с двумя воркерами без Off-Heap памяти?
2. Сравнить полученый ответ с реальными значениями.

<details>
    <summary>Нужна помощь?</summary>

    Ответы внизу страницы
</details>

### Off-Heap память для драйвера

Если воркеры могут иметь off-heap память, то логично предположить, что и драйвер имеет off-heap память тоже. Подобные рассуждения не далеки от истины, и действительно Off-Heap память для драйвера также можно сконфигурировать:

- если Spark запущен в **_кластерном_** режиме, то необходимо установить значение настройки `spark.driver.memoryOverhead` (`384MB` по умолчанию);
- если Spark запущен в **_клиентском_** режиме, то необходимо установить значение настройки `spark.yarn.am.memoryOverhead` (`384MB` по умолчанию).

PySpark запускается в **_клиентском_** режиме, поэтому будет использоваться настройка `spark.yarn.am.memoryOverhead`.

Для эксперимента установим `600MB` в качестве объема Off-Heap памяти для драйвера:

In [ ]:
spark = (
    SparkSession
        .builder
        .appName("Yarn")
        .master("yarn")
        .config("spark.executor.instances", 2)
        .config("spark.yarn.am.memory", "512m")
        .config("spark.yarn.am.memoryOverhead", "600m")
        .config("spark.executor.memory", "1g")
        .config("spark.executor.memoryOverhead", "0")
        .getOrCreate()
)
sc = spark.sparkContext

Общий объем памяти драйвера будет равен: $ 512MB (spark.yarn.am.memory) + 600MB (spark.yarn.am.memoryOverhead) = 1112MB $, что превышает размер слота Yarn (`1024MB`), а следовательно Yarn будет вынужен выделить два слота под драйвер.

Тогда общий объем памяти выделенный приложению будет равен:

Объем памяти драйвера: $ 512MB (spark.yarn.am.memory) + 600MB (spark.yarn.am.memoryOverhead) = 1112MB $

Число слотов для драйвера: $ ceil(1112 / 1024) = 2 $

Объем памяти воркеров: $ 1024MB (spark.executor.memory) + 0MB (spark.executor.memoryOverhead) = 1024MB $

Число слотов для воркеров: $ 2 (spark.executor.instances) $

Итого: $ (2 \space слота \space драйвера + 2 \space слота \space воркеров) \times 1024 = 4096MB $

In [ ]:
! source ~/.bash_aliases && \
yarn node -all -list -showDetails 2>/dev/null | grep 'Allocated Resources'

In [ ]:
spark.stop()

Полученное значение совпадает с реальным значением.

### Задание

1. Вычислить объем памяти для приложения с 3 воркерами.
    - Убрать Off-Heap значение для драйвера,
    - Установить 1GB в качестве Off-Heap значение для воркеров,
    - Оставить остальные значения по умолчанию.
1. Сравнить вычисленное значение с реальным значением.

<details>
    <summary>Нужна помощь?</summary>

    Ответ на странице ниже.
</details>

## Изменить минимальный объем слота

In [ ]:
spark.stop()

Создадим бэкап конфига:

In [ ]:
! cp -v $YARN_CONF_DIR/{,bkp_}yarn-site.xml

Минимальный объем слота будет равняться `64MB`:

In [ ]:
! sed -i '/yarn.scheduler.minimum-allocation-mb.value/s,1024,64,' $YARN_CONF_DIR/yarn-site.xml

Необходимо перезапустить Hadoop для активации новой конфигурации:

In [ ]:
! source ~/.bash_aliases && \
docker compose restart hadoop

Необходимо дождаться, когда Hadoop запустится: в колонке `STATUS` должно появиться значение `healthy`:

In [ ]:
! source ~/.bash_aliases && \
docker compose ps hadoop

In [ ]:
spark = (
    SparkSession
        .builder
        .appName("Yarn")
        .master("yarn")
        .config("spark.executor.instances", 2)
        .config("spark.yarn.am.memory", "512m")
        .config("spark.yarn.am.memoryOverhead", "448m") # 512 - 64 = 448
        .config("spark.executor.memory", "1g")
        .config("spark.executor.memoryOverhead", "512m")
        .getOrCreate()
)
sc = spark.sparkContext

В запущенном приложении все числа подобраны таким образом, что они без остатка делятся на 64MB (минимальный размер слота после перезапуска). Следовательно, выделенная память будет суммой значений:

$$
    512MB (spark.yarn.am.memory) + 448 (spark.yarn.am.memoryOverhead) + 2 (spark.executor.instances) \times (1024MB (spark.executor.memory) + 512MB (spark.executor.memoryOverhead)) = 4032MB
$$

In [ ]:
! source ~/.bash_aliases && \
yarn node -all -list -showDetails 2>/dev/null | grep 'Allocated Resources'

In [ ]:
spark.stop()

Восстановление оригинальной конфигурации Yarn:

In [ ]:
! cp -v $YARN_CONF_DIR/{bkp_,}yarn-site.xml

Необходимо перезапустить Hadoop для активации настроек:

In [ ]:
! source ~/.bash_aliases && \
docker compose restart hadoop

Необходимо дождаться, когда Hadoop запустится: в колонке `STATUS` должно появиться значение `healthy`:

In [ ]:
! source ~/.bash_aliases && \
docker compose ps hadoop

In [ ]:
! curl -s http://${RESOURCEMANAGER_HOST}:8088/conf | grep 'yarn.scheduler.minimum-allocation-mb' | xmllint --format -

In [ ]:
spark.stop()

## Выделение памяти больше допустимого

In [ ]:
spark.stop()

Максимально можно выделить не больше 3GB

In [ ]:
! curl -s http://${RESOURCEMANAGER_HOST}:8088/conf | grep 'yarn.scheduler.maximum-allocation-mb' | xmllint --format -

Запустим приложение, в котором запрашиваемый размер памяти воркера 4GB:

In [ ]:
try:
    spark = (
        SparkSession
            .builder
            .appName("Yarn")
            .master("yarn")
            .config("spark.executor.instances", 1)
            .config("spark.executor.memory", "3g")
            .config("spark.executor.memoryOverhead", "512m")
            .getOrCreate()
    )
    sc = spark.sparkContext
except Exception as e:
    print(e, file=sys.stderr)

Ожидаемо, возникла ошибка, т.к. приложению необходимо было выделить 4ГБ на один воркер (почему 4ГБ?).

Для решения этой проблемы необходимо увеличить значение `yarn.scheduler.maximum-allocation-mb`:

Создадим бэкап конфигурации Yarn:

In [ ]:
! cp -v $YARN_CONF_DIR/{,bkp_}yarn-site.xml

Установим максимальный объем памяти для разового выделения в `4096MB (4GB)`:

In [ ]:
! sed -i '/yarn.scheduler.maximum-allocation-mb.value/s,3072,4096,' $YARN_CONF_DIR/yarn-site.xml

Перезапуск Hadoop для активации новой конфигурации:

In [ ]:
! source ~/.bash_aliases && \
docker compose restart hadoop

Необходимо дождаться, когда Hadoop запустится: в колонке `STATUS` должно появиться значение `healthy`:

In [ ]:
! source ~/.bash_aliases && \
docker compose ps hadoop

In [ ]:
spark = (
    SparkSession
        .builder
        .appName("Yarn")
        .master("yarn")
        .config("spark.executor.instances", 1)
        .config("spark.executor.memory", "3g")
        .config("spark.executor.memoryOverhead", "512m")
        .getOrCreate()
)
sc = spark.sparkContext

Приложение запустилось. При этом несмотря на то, что максимально допустимый объем был установлен в значение `4096MB`, приложение получило `5120MB`:

In [ ]:
! source ~/.bash_aliases && \
yarn node -all -list -showDetails 2>/dev/null | grep 'Allocated Resources'

Связано это с тем, что ограчение устанавливается на единицу запроса, а всего приложение выполнило два запроса:

1. Один запрос на выделение ресурсов драйверу,
2. Один запрос на выделение ресурсов единственному воркеру (`spark.executor.instances`).

In [ ]:
spark.stop()

In [ ]:
! cp -v $YARN_CONF_DIR/{bkp_,}yarn-site.xml

Перезапуск Hadoop для активации новой конфигурации:

In [ ]:
! source ~/.bash_aliases && \
docker compose restart hadoop

Необходимо дождаться, когда Hadoop запустится: в колонке `STATUS` должно появиться значение `healthy`:

In [ ]:
! source ~/.bash_aliases && \
docker compose ps hadoop

In [ ]:
! curl -s http://${RESOURCEMANAGER_HOST}:8088/conf | grep 'yarn.scheduler.maximum-allocation-mb' | xmllint --format -

# Кластерный режим

При работе с Apache Spark через ноутбук автоматически выбирается **_клиентский режим_**: драйвер находится на локальном компьютере, а в облаке находится Application Master, необходимый для управления контейнерами Yarn для воркеров.

Для запуска приложения Spark в **_кластерном режиме_** необходимо использовать `spark-submit`.

Команда `spark-sumbit` запускает приложение в кластерном режиме, а значит будет использовать `spark.driver.memory` и `spark.driver.memoryOverhead` для запроса ресурсов.

Для примера запустим приложение, указав сразу два набора параметров:

- клиентский режим:
    - `spark.yarn.am.memory=2g`,
    - `spark.yarn.am.memoryOverhead=600m`.
- кластерный режим:
    - `--driver-memory 1g` (эта настройка аналогична `spark.driver.memory`),
    - `spark.driver.memoryOverhead=500m`.

В итоге должны примениться параметры кластерного режима:

In [ ]:
! /usr/local/spark/bin/spark-submit --class org.apache.spark.examples.SparkPi \
    --master yarn \
    --deploy-mode cluster \
    --driver-memory 1g \
    --conf spark.driver.memoryOverhead=500m \
    --conf spark.yarn.am.memory=2g \
    --conf spark.yarn.am.memoryOverhead=600m \
    --executor-memory 1g \
    --conf spark.executor.memoryOverhead=700m \
    --num-executors 2 \
    /usr/local/spark/examples/jars/spark-examples*.jar \
    10

Результат вычисления печается в стандартный поток вывода:

In [ ]:
! source ~/.bash_aliases && HOST=hadoop \
execute cat /opt/hadoop/logs/userlogs/$( \
    yarn app -list -appStates ALL |& awk '/org.apache.spark.examples.SparkPi/{print $1}' | sort -r | head -n 1 \
)/$( \
    yarn applicationattempt -list $( \
        yarn app -list -appStates ALL |& awk '/org.apache.spark.examples.SparkPi/{print $1}' | sort -r | head -n 1 \
    ) |& awk '/container_/{print $3}' \
)/stdout

В выводе команды выше можно увидеть, что приложение запросило `1524MB` для драйвера (Application Master): `Will allocate AM container, with 1524 MB memory including 500 MB overhead`, что соответствует теоретическим числам:

$$
1024MB (spark.driver.memory) + 500MB (spark.driver.memoryOverhead) = 1524MB
$$

Если бы приложение было запущено в клиентском режиме, то для Application Master контейнера было бы запрошено `2648MB`:

$$
2048MB (spark.yarn.am.memory) + 600MB (spark.yarn.am.memoryOverhead) = 2648MB
$$

Для воркеров было запрошено 2 контейнера (`--num-executors`) по `1724MB`:

$$
1024MB (--executor-memory) + 700MB (spark.executor.memoryOverhead) = 1724MB
$$

Команда ниже показывает, что ожидаемое значение соответствует реальному:

In [ ]:
! source ~/.bash_aliases && HOST=hadoop \
execute cat /opt/hadoop/logs/userlogs/$( \
    yarn app -list -appStates ALL |& awk '/org.apache.spark.examples.SparkPi/{print $1}' | sort -r | head -n 1 \
)/$( \
    yarn applicationattempt -list $( \
        yarn app -list -appStates ALL |& awk '/org.apache.spark.examples.SparkPi/{print $1}' | sort -r | head -n 1 \
    ) |& awk '/container_/{print $3}' \
)/stderr |& grep 'YarnAllocator: Will request'

В выводе можно увидеть:

- запрошено 2 контейнера: `Will request 2 executor container(s) ...`,
- на один контейнер `1724MB`: `Will request 2 executor container(s) ... each with 1 core(s) and 1724 MB memory`.

## Выводы

Очень важно понимать как режимы запуска приложения отличаются друг от друга и при каком режиме какие настройки используются. Разница в настройках заключатся только в том, какие настройки используются для запуска контейнера Application Master в кластере. В клиентском режиме драйвер находится на локальной машине программиста, а поэтому большой Application Master контейнер не нужен: его задача только следить за контейнерами воркеров и перезапускать их при необходимости. В кластерном режиме Application Master контейнер используется как для перезапуска контейнеров воркеров, так и для формирования планов запросов, а поэтому нужно больше ресурсов.

## Задание

1. Запустить приложение в кластерном режиме:
    - Память для драйвера: 768MB,
    - Off-Heap память для драйвера отсутствует,
    - Память для воркеров: 512MB,
    - Off-Heap память для воркеров: 200MB,
    - Число воркеров: 2.
1. Вычислить сколько памяти займет приложение с указанной конфигурацией;
1. Убедиться, что реальное значение памяти совпдает с ожидаемым значением;

<details>
    <summary>Нужна помощь?</summary>

    Ответы внизу страницы
</details>

# Вывод

Apache Spark стремится выполнять бОльшую часть работы в памяти, поэтому очень важно понимать, как управлять доступной памятью. Yarn предлагает простой подход к управлению ресурсами через слоты, но в свою очередь Spark может удивить неподготовленного программиста при выделении памяти сверх установленных значений, например, из-за настроек Off-Heap или разных режимов работы (client, cluster). К вопросу настройки памяти в каждом приложении стоит подходить индивидуально, следить за расходом памяти в реальной эксплуатации и подстраивать значения приложения в соответствии с полученными из реальной эксплуатации цифрами.

# Ответы

## Ответы на задания по созданию воркеров

1. Запустить Apache Spark приложение с настройками по умолчанию. Сколько контейнеров Yarn выделяет такому приложению?

<details>
    <summary>Ответ</summary>

    Три контейнера: 1 для драйвера и 2 (почему?) для воркеров

    Запустить приложение:
```python
spark = (
    SparkSession
        .builder
        .appName("Yarn Default")
        .master("yarn")
        .getOrCreate()
)
```

    Проверить контейнеры:
```bash
! source ~/.bash_aliases && \
yarn container -list $( \
    yarn applicationattempt -list $( \
        yarn application -appTypes SPARK -appStates RUNNING -list | awk '/application_/{print $1}' \
    ) | awk '/appattempt_/{print $1}' \
)
```

    Остановить приложение:
```python
spark.stop()
```
</details>

2. Запустить Apache Spark приложение с динамическим выделением ресурсов в соответствии со следующими требованиями:
    - в ждушем режиме не должно быть ниодного контейнера для воркеров,
    - максимальное число контейнеров 3,
    - время работы контейнера в ждущем режиме две минуты.

<details>
    <summary>Ответ</summary>

    Запустить приложение:
```python
spark = (
    SparkSession
        .builder
        .appName("Yarn Dymanic Workers")
        .master("yarn")
        .config("spark.dynamicAllocation.enabled", True)
        .config("spark.dynamicAllocation.minExecutors", 0)
        .config("spark.dynamicAllocation.maxExecutors", 3)
        .config("spark.dynamicAllocation.executorIdleTimeout", 120)
        .getOrCreate()
)
```

    Проверить контейнеры:

```bash
! source ~/.bash_aliases && \
yarn container -list $( \
    yarn applicationattempt -list $( \
        yarn application -appTypes SPARK -appStates RUNNING -list | awk '/application_/{print $1}' \
    ) | awk '/appattempt_/{print $1}' \
)
```

    Подождать 2 минуты и проверить контейнеры снова:

```bash
! source ~/.bash_aliases && sleep 120 && \
yarn container -list $( \
    yarn applicationattempt -list $( \
        yarn application -appTypes SPARK -appStates RUNNING -list | awk '/application_/{print $1}' \
    ) | awk '/appattempt_/{print $1}' \
)
```

    Остановить приложение
 
```python
spark.stop()
```
</details>

3. Запустить приложение с динамическим выделением ресурсов из предыдущего шага. При этом время жизни воркеров в ждущем режиме с закешированными данными должно не превышать 2 минуты 30 секунд.

<details>
    <summary>Ответ</summary>

    Запустить приложение:
```python
spark = (
    SparkSession
        .builder
        .appName("Yarn Dymanic Workers")
        .master("yarn")
        .config("spark.dynamicAllocation.enabled", True)
        .config("spark.dynamicAllocation.minExecutors", 0)
        .config("spark.dynamicAllocation.maxExecutors", 3)
        .config("spark.dynamicAllocation.executorIdleTimeout", 120)
        .config("spark.dynamicAllocation.cachedExecutorIdleTimeout", 150)
        .getOrCreate()
)
```

    Проверить контейнеры:

```bash
! source ~/.bash_aliases && \
yarn container -list $( \
    yarn applicationattempt -list $( \
        yarn application -appTypes SPARK -appStates RUNNING -list | awk '/application_/{print $1}' \
    ) | awk '/appattempt_/{print $1}' \
)
```

    Закешировать данные:
```python
from pyspark import StorageLevel

spark.range(1, 10000, 1, 1).toDF("id").persist(StorageLevel.MEMORY_ONLY)
```

    Подождать 2 минуты и проверить контейнеры снова:

```bash
! source ~/.bash_aliases && sleep 120 && \
yarn container -list $( \
    yarn applicationattempt -list $( \
        yarn application -appTypes SPARK -appStates RUNNING -list | awk '/application_/{print $1}' \
    ) | awk '/appattempt_/{print $1}' \
)
```
    Подождать 30 секунд и проверить контейнеры снова (должно остаться два контейнера: воркер с кешем и драйвер):

```bash
! source ~/.bash_aliases && sleep 30 && \
yarn container -list $( \
    yarn applicationattempt -list $( \
        yarn application -appTypes SPARK -appStates RUNNING -list | awk '/application_/{print $1}' \
    ) | awk '/appattempt_/{print $1}' \
)
```

    Остановить приложение:
 
```python
spark.stop()
```
</details>

## Ответы на задания по определению объема выделенной памяти

1. Аналитически определить сколько памяти будет выделено на приложение с двумя воркерами.

<details>
    <summary>Ответ</summary>

$ 1024MB (spark.driver.memory) + 2 \space (spark.executor.instances) \times 1024 (минимальный \space размер \space слота) \times 2 (слота) = 5120MB $

</details>

2. Запустить приложение с двумя воркерами. Проверить, что теоретические и практические значения совпадают.

<details>
    <summary>Ответ</summary>

    Запустить приложение:

```python
spark = (
    SparkSession
        .builder
        .appName("Yarn")
        .master("yarn")
        .config("spark.executor.instances", 2)
        .config("spark.yarn.am.memory", "512m")
        .config("spark.executor.memory", "1g")
        .getOrCreate()
)
sc = spark.sparkContext
```

    Проверить объем занимаемых ресуров:
```bash
! source ~/.bash_aliases && \
yarn node -all -list -showDetails 2>/dev/null | grep 'Allocated Resources'
```
    Остановить приложение:
```python
spark.stop()
```
</details>

### Ответы на задание по определению объема памяти для приложения без Off-Heap памяти

1. Аналитически вычислить объем оперативной памяти для приложения с двумя воркерами без Off-Heap памяти?
2. Сравнить полученый ответ с реальными значениями.

<details>
    <summary>Ответ</summary>

Минимальный размер слота: `1024MB`

Для одного воркера будет выделен один слот, т.к. $1024MB (spark.executor.memory) + 0MB (spark.executor.memoryOverhead) = 1024MB == минимальный \space размер \space слота$

Всего выделится 3 слота: один на драйвер, два на воркеры

В итоге будет выделено: $ 3 \space слота \times 1024MB = 3072MB $

    Запустить приложение без Off-Heap памяти:
```python
spark = (
    SparkSession
        .builder
        .appName("Yarn")
        .master("yarn")
        .config("spark.executor.instances", 3)
        .config("spark.yarn.am.memory", "512m")
        .config("spark.executor.memory", "1g")
        .config("spark.executor.memoryOverhead", "0")
        .getOrCreate()
)
```
    Проверить объем выделенной памяти
```bash
! source ~/.bash_aliases && \
yarn logs -am -1 -log_files stderr -applicationId $( \
    yarn application -appTypes SPARK -appStates RUNNING -list |& awk '/application_/{print $1}' \
) |& grep 'YarnAllocator: Will request'
```

```python
spark.stop()
```
</details>

## Ответ на задание для определения памяти для драйвера без Off-Heap памяти

1. Вычислить объем памяти для приложения с 3 воркерами.
    - Убрать Off-Heap значение для драйвера,
    - Установить 1GB в качестве Off-Heap значение для воркеров,
    - Оставить остальные значения по умолчанию.
1. Сравнить вычисленное значение с реальным значением.

<details>
    <summary>Ответ</summary>

Минимальный размер слота: `1024MB`

Для одного воркера будет выделено два слота, т.к. $1024MB (spark.executor.memory) + 1024MB (spark.executor.memoryOverhead) = 2048MB == 2 \times минимальный \space размер \space слота$

Для драйвера будет выделен один слот, т.к. $512MB (spark.yarn.am.memory) + 0MB (spark.yarn.am.memoryOverhead) = 512MB $, а Yarn может выделить за раз слот не меньше `1024MB`

Всего выделится 7 слотов: один на драйвер, шесть на воркеры

В итоге будет выделено: $ 7 \space слотов \times 1024MB = 7168MB $

    Запустить приложение без Off-Heap памяти:
```python
spark = (
    SparkSession
        .builder
        .appName("Yarn")
        .master("yarn")
        .config("spark.executor.instances", 3)
        .config("spark.yarn.am.memoryOverhead", "0")
        .config("spark.executor.memoryOverhead", "1g")
        .getOrCreate()
)
```
    Проверить объем выделенной памяти
```bash
! source ~/.bash_aliases && \
yarn logs -am -1 -log_files stderr -applicationId $( \
    yarn application -appTypes SPARK -appStates RUNNING -list |& awk '/application_/{print $1}' \
) |& grep 'YarnAllocator: Will request'
```

```python
spark.stop()
```
</details>

## Ответ на задания по запуску приложения в кластерном режиме

1. Запустить приложение в кластерном режиме:
    - Память для драйвера: 768MB,
    - Off-Heap память для драйвера отсутствует,
    - Память для воркеров: 512MB,
    - Off-Heap память для воркеров: 200MB,
    - Число воркеров: 2.
1. Вычислить сколько памяти займет приложение с указанной конфигурацией;
1. Убедиться, что реальное значение памяти совпдает с ожидаемым значением;

<details>
    <summary>Ответ</summary>

Всего приложение займет 3 слота: один слот на драйвер и два слота на воркеры (почему?). Тогда всего будет выделено памяти: $ 3 * 1024MB (минимальный \space размер \space слота) = 3072MB $

    Запуск приложения
```bash
! /usr/local/spark/bin/spark-submit --class org.apache.spark.examples.SparkPi \
    --master yarn \
    --deploy-mode cluster \
    --driver-memory 768m \
    --conf spark.driver.memoryOverhead=0 \
    --executor-memory 512m \
    --conf spark.executor.memoryOverhead=200m \
    --num-executors 2 \
    /usr/local/spark/examples/jars/spark-examples*.jar \
    10
```

В выводе можно обнаружить строку `Will allocate AM container, with 768 MB memory including 0 MB overhead`, т.е. на драйвер будет выделено `768MB`, что полностью умещается в один слот (`1024MB`)

В логах приложения можно найти найти сколько контейнеров запрашивало приложение:

```bash
! source ~/.bash_aliases && HOST=hadoop \
execute cat /opt/hadoop/logs/userlogs/$( \
    yarn app -list -appStates ALL |& awk '/org.apache.spark.examples.SparkPi/{print $1}' | sort -r | head -n 1 \
)/$( \
    yarn applicationattempt -list $( \
        yarn app -list -appStates ALL |& awk '/org.apache.spark.examples.SparkPi/{print $1}' | sort -r | head -n 1 \
    ) |& awk '/container_/{print $3}' \
)/stderr |& grep 'YarnAllocator: Will request'
```

Контейнер Application Master запросил 2 контейнера по `712MB`: `Will request 2 executor container(s) for  ResourceProfile Id: 0, each with 1 core(s) and 712 MB memory`. Можно заключить:

- приложение запросило на один контейнер значение `712MB`, которое состоит из:
    - `--executor-memory 512m`
    - `spark.executor.memoryOverhead=200m`
- один контейнер полностью умещается в один слот (`1024MB`)
- всего приложение (драйвер и 2 воркера) займет `3072MB`

</details>